In [1]:
%matplotlib notebook
%reload_ext autoreload
%autoreload 2

In [5]:
import pickle
import sys
sys.path.insert(0, '../../preprocess')
import vectorizer

with open('../../preprocess/cohendata_dedup_19995.p', 'rb') as f:
    u = pickle._Unpickler(f)
    u.encoding = 'latin1'
    lang_1 = u.load()

In [9]:
import re

joined = [[lang_1.idx2word[y] for y in sent] for sent in lang_1.X]
joined = [list(filter(lambda x: '\n' not in x and '[0]' not in x, sent)) for sent in joined]
joined = [' '.join(sent) for sent in joined]
joined = [re.sub(r'\. qqq \.', '.', sent) for sent in joined]
joined = [sent.split(' ') for sent in joined]

In [16]:
from itertools import groupby
new_sents = []
for sent in joined :
    sent = [x for x in sent if x not in ['<SOS>', '<EOS>'] and '\n' not in x]
    sents = [list(group) for k, group in groupby(sent, lambda x: x == '.' or x == '?') if not k]
    new_sents.append(sents)

In [19]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()

In [20]:
process_sents = []
import re
stops = stopwords.words('english')
for doc in new_sents :
    changes = []
    for sent in doc :
        sent = [re.sub('[^a-zA-Z0-9]', '', x).strip() for x in sent]
        sent = [x for x in sent if len(x) > 0]
        sent = [word for word in sent if word not in stops]
        sent = [lmtzr.lemmatize(word) for word in sent]
        changes.append(sent)
    process_sents.append(changes)

In [28]:
data = pickle.load(open('data_pico_new.p', 'rb'))

In [30]:
idx_cohen = []
word2idx = data['word2idx']
for doc in process_sents :
    idxs = []
    for sent in doc :
        nline = []
        for word in sent :
            if any(char.isdigit() for char in word) :
                nline.append(word2idx['qqq'])
            elif word in word2idx :
                nline.append(word2idx[word])
            else :
                nline.append(word2idx['unk'])
        idxs.append(nline)
    idx_cohen.append(idxs)

In [34]:
data['cohen'] = idx_cohen

In [37]:
pickle.dump(data, open('data_pico_cohen.p', 'wb'))